# 1 Cargamos librerias

In [15]:
# Importaciones
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import time
import joblib
import mlflow
from mlflow.tracking import MlflowClient

from loguru import logger

# Importar módulos del paquete product_development
from product_development.config import (
    TARGET, FEATURES, CATEGORICAL_VARS, RAW_DATA_DIR, MODELS_DIR,
    FEATURE_PIPELINE_FILE, PIPELINE_FILE, TRAIN_TEST_SPLIT_RATIO, RANDOM_STATE,
    MLFLOW_EXPERIMENT_NAME, MLFLOW_MODEL_NAME, MLFLOW_TRACKING_URI,
    MLFLOW_CHAMPION_ALIAS, MLFLOW_CHALLENGER_ALIAS
)
from product_development.dataset import load_raw_data, prepare_dataset, temporal_train_test_split
from product_development.features import load_feature_pipeline, transform_features
from product_development.modeling.train import (
    get_model_configurations, train_and_evaluate_models,
    create_full_pipeline, train_final_model, save_pipeline,
    setup_mlflow, register_model_to_mlflow, get_champion_model,
    compare_with_champion, promote_challenger_to_champion, get_experiment_summary,
    calculate_metrics
)

Configuración de modelos, logging y rutas

In [16]:
# 2. Configuración de modelos y MLflow
# Modo: "fast" (rápido, 4 modelos) o "full" (completo, 15 modelos)
TRAINING_MODE = "fast"  # Cambiar a "full" para evaluación completa
USE_MLFLOW = True  # Habilitar/deshabilitar tracking de MLflow

# Configurar MLflow
if USE_MLFLOW:
    experiment_id = setup_mlflow(MLFLOW_EXPERIMENT_NAME, MLFLOW_TRACKING_URI)
    print(f"MLflow Experiment ID: {experiment_id}")
    print(f"MLflow Tracking URI: {MLFLOW_TRACKING_URI}")

models_configurations = get_model_configurations(mode=TRAINING_MODE)

print(f"\nModo de entrenamiento: {TRAINING_MODE}")
print(f"Total de modelos a evaluar: {len(models_configurations)}")
print("\nModelos configurados:")
for name in models_configurations.keys():
    print(f"  - {name}")

2025-11-29 10:24:07.206 | INFO     | product_development.modeling.train:setup_mlflow:155 - MLflow configurado - Experimento: sales_prediction
MLflow Experiment ID: 781360344278088554
MLflow Tracking URI: mlruns

Modo de entrenamiento: fast
Total de modelos a evaluar: 4

Modelos configurados:
  - LinearRegression
  - RandomForest
  - GradientBoosting
  - XGBoost


# 3 Entrenamiento y seleccion de modelos


In [17]:
# 3. Cargar y preparar datos usando los módulos del paquete
raw_data = load_raw_data(RAW_DATA_DIR / "train.csv")
dataset = prepare_dataset(raw_data)

print(f"Total registros: {len(dataset)}")

# Split temporal 80/20
X_train, X_val, y_train, y_val = temporal_train_test_split(dataset, TRAIN_TEST_SPLIT_RATIO)

print(f"Train: {X_train.shape}, Val: {X_val.shape}")

# Cargar pipeline de preprocesamiento
preproc_pipeline = load_feature_pipeline(FEATURE_PIPELINE_FILE)

# Ajustar pipeline con datos de entrenamiento
preproc_pipeline.fit(X_train, y_train)

# Transformar datos
X_train_proc = transform_features(preproc_pipeline, X_train)
X_val_proc = transform_features(preproc_pipeline, X_val)

print(f"Shape X_train_proc: {X_train_proc.shape}")
print(f"Shape X_val_proc: {X_val_proc.shape}")

2025-11-29 10:24:07.217 | INFO     | product_development.dataset:load_raw_data:49 - Cargando datos crudos desde C:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\data\raw\train.csv
2025-11-29 10:24:07.533 | INFO     | product_development.dataset:load_raw_data:51 - Cargados 913000 registros
2025-11-29 10:24:07.534 | INFO     | product_development.dataset:add_temporal_features:69 - Agregando características temporales
2025-11-29 10:24:07.669 | INFO     | product_development.dataset:convert_categorical_types:92 - Convirtiendo columnas categóricas a tipo object
Total registros: 913000
2025-11-29 10:24:07.731 | INFO     | product_development.dataset:temporal_train_test_split:140 - Dividiendo datos con 80% para entrenamiento
2025-11-29 10:24:07.533 | INFO     | product_development.dataset:load_raw_data:51 - Cargados 913000 registros
2025-11-29 10:24:07.534 | INFO     | product_development.dataset:add_temporal_features:69 - Agregando características temporales
2025

c:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\notebooks\..\product_development\transformers.py:135: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[var] = X[var].fillna(self.fill_value).astype(str)
c:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\notebooks\..\product_development\transformers.py:135: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[var] = X[var].fillna(self.fill_value).astype(str)


2025-11-29 10:24:09.006 | INFO     | product_development.features:transform_features:126 - Transformando 730400 registros


c:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\notebooks\..\product_development\transformers.py:135: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[var] = X[var].fillna(self.fill_value).astype(str)
c:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\notebooks\..\product_development\transformers.py:135: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[var] = X[var].fillna(self.fill_value).astype(str)


2025-11-29 10:24:09.832 | INFO     | product_development.features:transform_features:126 - Transformando 182600 registros
Shape X_train_proc: (730400, 5)
Shape X_val_proc: (182600, 5)
Shape X_train_proc: (730400, 5)
Shape X_val_proc: (182600, 5)


c:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\notebooks\..\product_development\transformers.py:135: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[var] = X[var].fillna(self.fill_value).astype(str)
c:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\notebooks\..\product_development\transformers.py:135: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[var] = X[var].fillna(self.fill_value).astype(str)


In [18]:
# 4. Entrenamiento y evaluación de modelos con MLflow tracking
start = time.time()

results, best_model_name, best_model = train_and_evaluate_models(
    X_train_proc, y_train,
    X_val_proc, y_val,
    models_configurations,
    use_mlflow=USE_MLFLOW
)

elapsed_time = round(time.time() - start, 2)

print(f"\nTiempo total de entrenamiento: {elapsed_time} segundos")
print(f"\nResultados de todos los modelos:")
for model_name, metrics in results.items():
    print(f"  {model_name}: RMSE={metrics['rmse']}, R2={metrics['r2']}")

print(f"\n🏆 Mejor modelo: {best_model_name}")
print(f"   RMSE: {results[best_model_name]['rmse']}")
print(f"   R2: {results[best_model_name]['r2']}")

2025-11-29 10:24:10.044 | INFO     | product_development.modeling.train:train_and_evaluate_models:245 - Iniciando entrenamiento y evaluación de modelos
2025-11-29 10:24:10.052 | INFO     | product_development.modeling.train:setup_mlflow:155 - MLflow configurado - Experimento: sales_prediction


Entrenando modelos:   0%|          | 0/4 [00:00<?, ?it/s]2025/11/29 10:24:10 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/29 10:24:10 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


2025-11-29 10:24:10.053 | INFO     | product_development.modeling.train:train_and_evaluate_models:252 - Entrenando modelo: LinearRegression


2025/11/29 10:24:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Entrenando modelos:  25%|██▌       | 1/4 [00:03<00:09,  3.31s/it]

2025-11-29 10:24:13.362 | INFO     | product_development.modeling.train:train_and_evaluate_models:281 - Modelo LinearRegression - RMSE: 30.0452, R2: 0.0931
2025-11-29 10:24:13.362 | INFO     | product_development.modeling.train:train_and_evaluate_models:252 - Entrenando modelo: RandomForest


...

2025-11-29 10:24:20.627 | INFO     | product_development.modeling.train:train_and_evaluate_models:281 - Modelo RandomForest - RMSE: 27.4841, R2: 0.2411
2025-11-29 10:24:20.644 | INFO     | product_development.modeling.train:train_and_evaluate_models:252 - Entrenando modelo: GradientBoosting


2025/11/29 10:24:39 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/29 10:24:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/29 10:24:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Entrenando modelos:  75%|███████▌  | 3/4 [00:32<00:13, 13.04s/it]

2025-11-29 10:24:42.476 | INFO     | product_development.modeling.train:train_and_evaluate_models:281 - Modelo GradientBoosting - RMSE: 27.1293, R2: 0.2605
2025-11-29 10:24:42.481 | INFO     | product_development.modeling.train:train_and_evaluate_models:252 - Entrenando modelo: XGBoost


2025/11/29 10:24:43 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/29 10:24:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/29 10:24:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Entrenando modelos: 100%|██████████| 4/4 [00:36<00:00,  9.14s/it]

2025-11-29 10:24:46.599 | INFO     | product_development.modeling.train:train_and_evaluate_models:281 - Modelo XGBoost - RMSE: 26.9235, R2: 0.2717
2025-11-29 10:24:46.607 | INFO     | product_development.modeling.train:train_and_evaluate_models:301 - Mejor modelo: XGBoost con RMSE = 26.9235

Tiempo total de entrenamiento: 36.56 segundos

Resultados de todos los modelos:
  LinearRegression: RMSE=30.0452, R2=0.0931
  RandomForest: RMSE=27.4841, R2=0.2411
  GradientBoosting: RMSE=27.1293, R2=0.2605
  XGBoost: RMSE=26.9235, R2=0.2717

🏆 Mejor modelo: XGBoost
   RMSE: 26.9235
   R2: 0.2717


# 4. Importamos Pipeline de preproc y Agregamos Modelo Ganador


In [19]:
# 5. Crear pipeline completo con el mejor modelo
full_pipeline = create_full_pipeline(preproc_pipeline, best_model)

print("Pipeline completo creado:")
for step_name, step in full_pipeline.steps:
    print(f"  - {step_name}: {type(step).__name__}")

2025-11-29 10:24:46.614 | INFO     | product_development.modeling.train:create_full_pipeline:325 - Creando pipeline completo con modelo
Pipeline completo creado:
  - cat_missing_imputation: SimpleCategoricalImputer
  - num_median_imputation: MeanMedianImputer
  - cat_freq_encoder: CountFrequencyEncoder
  - dayofweek_mapper: Mapper
  - feature_scaler: MinMaxScaler
  - regressor: XGBRegressor


# 5. Re entrenamos pipelines con el Modelo Ganador


In [20]:
# 6. Re-entrenar con el dataset completo
X_full = dataset[FEATURES].copy()
y_full = dataset[TARGET].copy()

# Convertir tipos categóricos
for col in CATEGORICAL_VARS:
    X_full[col] = X_full[col].astype("O")

full_pipeline = train_final_model(full_pipeline, X_full, y_full)

print("Pipeline final entrenado correctamente.")

2025-11-29 10:24:46.740 | INFO     | product_development.modeling.train:train_final_model:357 - Entrenando modelo final con dataset completo


c:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\notebooks\..\product_development\transformers.py:135: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[var] = X[var].fillna(self.fill_value).astype(str)
c:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\notebooks\..\product_development\transformers.py:135: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[var] = X[var].fillna(self.fill_value).astype(str)
c:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\notebooks\..\product_d

2025-11-29 10:24:49.082 | INFO     | product_development.modeling.train:train_final_model:359 - Entrenamiento del modelo final completado
Pipeline final entrenado correctamente.


In [21]:
# 7. Guardar pipeline localmente
save_pipeline(full_pipeline, PIPELINE_FILE)
print(f"Pipeline completo guardado en: {PIPELINE_FILE}")

2025-11-29 10:24:49.102 | INFO     | product_development.modeling.train:save_pipeline:379 - Pipeline guardado en C:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\models\sales_pipeline.pkl
Pipeline completo guardado en: C:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\models\sales_pipeline.pkl


# 8. Registro en MLflow Model Registry y Gestión Champion/Challenger

In [22]:
# 8.1 Calcular métricas finales del pipeline completo
if USE_MLFLOW:
    # Predecir con el pipeline completo (usando datos sin transformar)
    X_val_raw = dataset.iloc[int(len(dataset) * TRAIN_TEST_SPLIT_RATIO):][FEATURES].copy()
    y_val_raw = dataset.iloc[int(len(dataset) * TRAIN_TEST_SPLIT_RATIO):][TARGET].copy()
    
    # Convertir tipos categóricos
    for col in CATEGORICAL_VARS:
        X_val_raw[col] = X_val_raw[col].astype("O")
    
    y_pred_final = full_pipeline.predict(X_val_raw)
    final_metrics = calculate_metrics(y_val_raw, y_pred_final)
    
    print("📊 Métricas finales del pipeline completo:")
    print(f"   RMSE: {final_metrics['rmse']}")
    print(f"   MAE: {final_metrics['mae']}")
    print(f"   R2: {final_metrics['r2']}")
    print(f"   MSE: {final_metrics['mse']}")

c:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\notebooks\..\product_development\transformers.py:135: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[var] = X[var].fillna(self.fill_value).astype(str)
c:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\notebooks\..\product_development\transformers.py:135: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[var] = X[var].fillna(self.fill_value).astype(str)


📊 Métricas finales del pipeline completo:
   RMSE: 24.1248
   MAE: 20.1097
   R2: 0.0946
   MSE: 582.0082


In [23]:
# 8.2 Comparar con el modelo Champion actual (si existe)
if USE_MLFLOW:
    is_better, champion_metrics = compare_with_champion(
        final_metrics, 
        model_name=MLFLOW_MODEL_NAME,
        metric_key="rmse"
    )
    
    if champion_metrics:
        print("\n📈 Comparación con el modelo Champion actual:")
        print(f"   Champion RMSE: {champion_metrics.get('rmse', 'N/A')}")
        print(f"   Challenger RMSE: {final_metrics['rmse']}")
        
        if is_better:
            print("\n✅ El nuevo modelo es MEJOR que el Champion actual!")
        else:
            print("\n⚠️ El nuevo modelo NO supera al Champion actual")
    else:
        print("\n📝 No hay modelo Champion registrado. Este será el primero.")

2025-11-29 10:24:49.448 | INFO     | product_development.modeling.train:setup_mlflow:155 - MLflow configurado - Experimento: sales_prediction
2025-11-29 10:24:49.489 | INFO     | product_development.modeling.train:compare_with_champion:516 - Champion rmse: 24.1248
2025-11-29 10:24:49.489 | INFO     | product_development.modeling.train:compare_with_champion:517 - Challenger rmse: 24.1248

📈 Comparación con el modelo Champion actual:
   Champion RMSE: 24.1248
   Challenger RMSE: 24.1248

⚠️ El nuevo modelo NO supera al Champion actual


In [24]:
# 8.3 Registrar modelo en MLflow Model Registry
if USE_MLFLOW:
    # Preparar input_example para la signature del modelo (muestra de datos sin transformar)
    input_example_df = X_val_raw.head(5)
    
    # Decidir si registrar como champion o challenger
    if champion_metrics is None or is_better:
        # Primera vez o mejor que el champion -> registrar como champion
        version = register_model_to_mlflow(
            pipeline=full_pipeline,
            model_name=MLFLOW_MODEL_NAME,
            best_model_name=best_model_name,
            metrics=final_metrics,
            register_as_champion=True,
            input_example=input_example_df
        )
        print(f"\n🏆 Modelo registrado como CHAMPION (versión {version})")
    else:
        # No es mejor -> registrar como challenger para comparación futura
        version = register_model_to_mlflow(
            pipeline=full_pipeline,
            model_name=MLFLOW_MODEL_NAME,
            best_model_name=best_model_name,
            metrics=final_metrics,
            register_as_champion=False,
            input_example=input_example_df
        )
        print(f"\n🥈 Modelo registrado como CHALLENGER (versión {version})")

2025/11/29 10:24:49 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


2025-11-29 10:24:49.522 | INFO     | product_development.modeling.train:setup_mlflow:155 - MLflow configurado - Experimento: sales_prediction


2025/11/29 10:24:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'sales_prediction_model' already exists. Creating a new version of this model...
Created version '2' of model 'sales_prediction_model'.
c:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\notebooks\..\product_development\modeling\train.py:429: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(model_name)
Registered model 'sales_prediction_model' already exists. Creating a new version of this model...
Created version '2' of model

2025-11-29 10:24:52.770 | INFO     | product_development.modeling.train:register_model_to_mlflow:425 - Pipeline registrado en MLflow: models:/m-517d5d587e7a46c3a9edb9c8b5a8843a
2025-11-29 10:24:52.800 | INFO     | product_development.modeling.train:register_model_to_mlflow:433 - Modelo registrado como versión: 2
2025-11-29 10:24:52.863 | INFO     | product_development.modeling.train:register_model_to_mlflow:446 - Modelo versión 2 marcado como 'challenger'

🥈 Modelo registrado como CHALLENGER (versión 2)


# 9. Resumen del Experimento MLflow

In [25]:
# 9.1 Obtener resumen de todos los experimentos
if USE_MLFLOW:
    experiment_df = get_experiment_summary(MLFLOW_EXPERIMENT_NAME)
    
    if not experiment_df.empty:
        print("📋 Resumen de todos los runs del experimento:")
        print(f"   Total de runs: {len(experiment_df)}")
        
        # Mostrar las métricas más importantes
        metric_cols = [col for col in experiment_df.columns if 'rmse' in col or 'r2' in col]
        display_cols = ['run_name', 'status'] + metric_cols
        available_cols = [col for col in display_cols if col in experiment_df.columns]
        
        display(experiment_df[available_cols].head(10))
    else:
        print("No hay runs registrados en el experimento")

2025-11-29 10:24:52.883 | INFO     | product_development.modeling.train:setup_mlflow:155 - MLflow configurado - Experimento: sales_prediction
📋 Resumen de todos los runs del experimento:
   Total de runs: 10
📋 Resumen de todos los runs del experimento:
   Total de runs: 10


,status,metrics.rmse,metrics.r2
0,FINISHED,24.1248,0.0946
1,FINISHED,26.9235,0.2717
2,FINISHED,27.1293,0.2605
3,FINISHED,27.4841,0.2411
4,FINISHED,30.0452,0.0931
5,FINISHED,24.1248,0.0946
6,FINISHED,26.9235,0.2717
7,FINISHED,27.1293,0.2605
8,FINISHED,27.4841,0.2411
9,FINISHED,30.0452,0.0931


In [26]:
# 9.2 Ver información del modelo registrado en Model Registry
if USE_MLFLOW:
    client = MlflowClient()
    
    try:
        # Obtener todas las versiones del modelo
        versions = client.search_model_versions(f"name='{MLFLOW_MODEL_NAME}'")
        
        print(f"\n📦 Modelo: {MLFLOW_MODEL_NAME}")
        print(f"   Total de versiones: {len(versions)}")
        
        # Mostrar información del champion actual
        try:
            champion_version = client.get_model_version_by_alias(MLFLOW_MODEL_NAME, MLFLOW_CHAMPION_ALIAS)
            champion_run = client.get_run(champion_version.run_id)
            print(f"\n🏆 Champion actual (versión {champion_version.version}):")
            print(f"   RMSE: {champion_run.data.metrics.get('rmse', 'N/A')}")
            print(f"   R2: {champion_run.data.metrics.get('r2', 'N/A')}")
        except:
            print("\n   No hay modelo Champion asignado")
        
        # Mostrar información del challenger actual
        try:
            challenger_version = client.get_model_version_by_alias(MLFLOW_MODEL_NAME, MLFLOW_CHALLENGER_ALIAS)
            challenger_run = client.get_run(challenger_version.run_id)
            print(f"\n🥈 Challenger actual (versión {challenger_version.version}):")
            print(f"   RMSE: {challenger_run.data.metrics.get('rmse', 'N/A')}")
            print(f"   R2: {challenger_run.data.metrics.get('r2', 'N/A')}")
        except:
            print("\n   No hay modelo Challenger asignado")
            
    except Exception as e:
        print(f"Error obteniendo información del modelo: {e}")


📦 Modelo: sales_prediction_model
   Total de versiones: 2

🏆 Champion actual (versión 1):
   RMSE: 24.1248
   R2: 0.0946

🥈 Challenger actual (versión 2):
   RMSE: 24.1248
   R2: 0.0946


In [27]:
# 9.3 (Opcional) Promover Challenger a Champion manualmente
# Descomenta las siguientes líneas si deseas promover manualmente un challenger a champion

# if USE_MLFLOW:
#     success = promote_challenger_to_champion(model_name=MLFLOW_MODEL_NAME)
#     if success:
#         print("✅ Challenger promovido a Champion exitosamente!")
#     else:
#         print("❌ Error al promover Challenger")

In [28]:
# 9.4 Instrucciones para visualizar MLflow UI
print("=" * 60)
print("🖥️  Para visualizar los experimentos en MLflow UI ejecuta:")
print("=" * 60)
print("\n   mlflow ui --backend-store-uri mlruns")
print("\n   Luego abre en tu navegador: http://localhost:5000")
print("\n" + "=" * 60)
print("📊 En la UI podrás ver:")
print("   - Todos los experimentos y runs")
print("   - Comparación de métricas entre modelos")
print("   - Hiperparámetros de cada modelo")
print("   - Model Registry con versiones Champion/Challenger")
print("=" * 60)

🖥️  Para visualizar los experimentos en MLflow UI ejecuta:

   mlflow ui --backend-store-uri mlruns

   Luego abre en tu navegador: http://localhost:5000

📊 En la UI podrás ver:
   - Todos los experimentos y runs
   - Comparación de métricas entre modelos
   - Hiperparámetros de cada modelo
   - Model Registry con versiones Champion/Challenger
